In [1]:
## This file implements neural networks and logistic regression on p11presabsSTCC_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 92.08% for combination data, and 83.21% for over-sampling data 
## after improvements and regularization.
## To improve accuracy, we use lasso to select important features and thus have new training and test datasets.
## However, with the new datasets, the accuracy becomes 90.62% for combination data and 83.97% for over-sampling data.
## For logistic regression, the accuracies are 97.03% for combination data and 90.84% for over-sampling data.
## For random forest, the accuracy is 97.03% for combination and 97.71% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.27% for combination data, and 94.76% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p11presabsSTCC_quant.csv')
df.shape

(255, 976)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.971625
1      0.591125
2      0.902750
3      1.169875
4      1.147000
5      1.112875
6      0.936500
7      0.707000
8      0.806375
9      0.921500
10     0.510000
11     0.958000
12     1.036750
13     0.427625
14     1.052250
15     1.079500
16     0.390375
17     0.385125
18     1.025500
19     0.225750
20     0.295875
21     0.348250
22     0.507375
23     1.114000
24     1.089625
25     0.163000
26     0.888125
27     0.198375
28     0.828250
29     1.153375
         ...   
225    0.975300
226    0.698300
227    0.933700
228    1.012100
229    1.104800
230    0.621000
231    0.941250
232    0.928500
233    0.902250
234    0.897000
235    1.047375
236    0.992625
237    0.901250
238    1.075750
239    0.758000
240    0.831750
241    0.891000
242    0.369125
243    0.772000
244    0.986750
245    0.975375
246    1.125250
247    0.828750
248    1.086250
249    0.854750
250    0.905000
251    0.572500
252    1.071000
253    1.000500
254    0.955125
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno,ST,CC
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.971625,5,5
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.591125,8,8
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.902750,5,5
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1.169875,5,5
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1.147000,5,5


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

1    218
0     37
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 976)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 975)

In [13]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno,ST,CC
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,8,8
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 974) (255,)


In [15]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 175), (1, 161)]


In [16]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 218), (1, 218)]


In [17]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [18]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [19]:
############# Fully-Connected Neural Network ################

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [21]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [22]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 235 samples, validate on 101 samples
Epoch 1/1000
235/235 [==============================] - 0s 625us/step - loss: 3.4332 - accuracy: 0.6511 - val_loss: 0.5741 - val_accuracy: 0.8119
Epoch 2/1000
235/235 [==============================] - 0s 151us/step - loss: 0.5128 - accuracy: 0.8128 - val_loss: 0.5930 - val_accuracy: 0.8515
Epoch 3/1000
235/235 [==============================] - 0s 125us/step - loss: 0.5725 - accuracy: 0.8383 - val_loss: 0.4951 - val_accuracy: 0.7129
Epoch 4/1000
235/235 [==============================] - 0s 136us/step - loss: 0.4448 - accuracy: 0.7957 - val_loss: 0.4630 - val_accuracy: 0.7228
Epoch 5/1000
235/235 [==============================] - 0s 157us/step - loss: 0.3975 - accuracy: 0.8511 - val_loss: 0.9061 - val_accuracy: 0.8812
Epoch 6/1000
235/235 [==============================] - 0s 197us/step - loss: 0.3653 - accuracy: 0.8723 - val_loss: 2.2030 - val_accuracy: 0.8812
Epoch 7/1000
235/235 [==============================] - 0s 129us/step - loss: 

235/235 [==============================] - 0s 124us/step - loss: 0.2130 - accuracy: 0.9362 - val_loss: 0.2860 - val_accuracy: 0.9010
Epoch 57/1000
235/235 [==============================] - 0s 116us/step - loss: 0.2148 - accuracy: 0.9362 - val_loss: 0.2850 - val_accuracy: 0.9208
Epoch 58/1000
235/235 [==============================] - 0s 118us/step - loss: 0.2180 - accuracy: 0.9362 - val_loss: 0.2839 - val_accuracy: 0.9208
Epoch 59/1000
235/235 [==============================] - 0s 131us/step - loss: 0.2164 - accuracy: 0.9319 - val_loss: 0.2837 - val_accuracy: 0.9208
Epoch 60/1000
235/235 [==============================] - 0s 126us/step - loss: 0.2142 - accuracy: 0.9319 - val_loss: 0.2828 - val_accuracy: 0.9208
Epoch 61/1000
235/235 [==============================] - 0s 128us/step - loss: 0.2100 - accuracy: 0.9362 - val_loss: 0.2804 - val_accuracy: 0.9208
Epoch 62/1000
235/235 [==============================] - 0s 134us/step - loss: 0.2119 - accuracy: 0.9362 - val_loss: 0.2834 - val_ac

Epoch 112/1000
235/235 [==============================] - 0s 102us/step - loss: 0.1789 - accuracy: 0.9404 - val_loss: 0.2455 - val_accuracy: 0.9208
Epoch 113/1000
235/235 [==============================] - 0s 92us/step - loss: 0.1794 - accuracy: 0.9404 - val_loss: 0.2567 - val_accuracy: 0.9109
Epoch 114/1000
235/235 [==============================] - 0s 97us/step - loss: 0.1832 - accuracy: 0.9404 - val_loss: 0.2438 - val_accuracy: 0.9208
Epoch 115/1000
235/235 [==============================] - 0s 97us/step - loss: 0.1790 - accuracy: 0.9362 - val_loss: 0.2489 - val_accuracy: 0.9307
Epoch 116/1000
235/235 [==============================] - 0s 153us/step - loss: 0.1796 - accuracy: 0.9404 - val_loss: 0.2429 - val_accuracy: 0.9208
Epoch 117/1000
235/235 [==============================] - 0s 288us/step - loss: 0.1794 - accuracy: 0.9404 - val_loss: 0.2423 - val_accuracy: 0.9208
Epoch 118/1000
235/235 [==============================] - 0s 453us/step - loss: 0.1795 - accuracy: 0.9404 - val_los

235/235 [==============================] - 0s 121us/step - loss: 0.1658 - accuracy: 0.9447 - val_loss: 0.2180 - val_accuracy: 0.9208
Epoch 168/1000
235/235 [==============================] - 0s 134us/step - loss: 0.1572 - accuracy: 0.9447 - val_loss: 0.2172 - val_accuracy: 0.9307
Epoch 169/1000
235/235 [==============================] - 0s 139us/step - loss: 0.1579 - accuracy: 0.9447 - val_loss: 0.2169 - val_accuracy: 0.9307
Epoch 170/1000
235/235 [==============================] - 0s 135us/step - loss: 0.1560 - accuracy: 0.9447 - val_loss: 0.2199 - val_accuracy: 0.9307
Epoch 171/1000
235/235 [==============================] - 0s 130us/step - loss: 0.1563 - accuracy: 0.9447 - val_loss: 0.2155 - val_accuracy: 0.9208
Epoch 172/1000
235/235 [==============================] - 0s 149us/step - loss: 0.1570 - accuracy: 0.9447 - val_loss: 0.2152 - val_accuracy: 0.9307
Epoch 173/1000
235/235 [==============================] - 0s 139us/step - loss: 0.1586 - accuracy: 0.9447 - val_loss: 0.2177 - 

235/235 [==============================] - 0s 205us/step - loss: 0.1418 - accuracy: 0.9447 - val_loss: 0.1967 - val_accuracy: 0.9307
Epoch 223/1000
235/235 [==============================] - 0s 156us/step - loss: 0.1432 - accuracy: 0.9447 - val_loss: 0.1967 - val_accuracy: 0.9307
Epoch 224/1000
235/235 [==============================] - 0s 129us/step - loss: 0.1399 - accuracy: 0.9447 - val_loss: 0.1979 - val_accuracy: 0.9307
Epoch 225/1000
235/235 [==============================] - 0s 102us/step - loss: 0.1404 - accuracy: 0.9447 - val_loss: 0.2061 - val_accuracy: 0.9406
Epoch 226/1000
235/235 [==============================] - 0s 123us/step - loss: 0.1420 - accuracy: 0.9447 - val_loss: 0.2003 - val_accuracy: 0.9307
Epoch 227/1000
235/235 [==============================] - 0s 127us/step - loss: 0.1443 - accuracy: 0.9447 - val_loss: 0.2081 - val_accuracy: 0.9406
Epoch 228/1000
235/235 [==============================] - 0s 106us/step - loss: 0.1417 - accuracy: 0.9489 - val_loss: 0.1950 - 

...

235/235 [==============================] - 0s 256us/step - loss: 0.0987 - accuracy: 0.9617 - val_loss: 0.1473 - val_accuracy: 0.9505
Epoch 774/1000
235/235 [==============================] - 0s 149us/step - loss: 0.0986 - accuracy: 0.9617 - val_loss: 0.1455 - val_accuracy: 0.9505
Epoch 775/1000
235/235 [==============================] - 0s 155us/step - loss: 0.0987 - accuracy: 0.9617 - val_loss: 0.1455 - val_accuracy: 0.9505
Epoch 776/1000
235/235 [==============================] - 0s 97us/step - loss: 0.0987 - accuracy: 0.9617 - val_loss: 0.1452 - val_accuracy: 0.9505
Epoch 777/1000
235/235 [==============================] - 0s 159us/step - loss: 0.0986 - accuracy: 0.9617 - val_loss: 0.1477 - val_accuracy: 0.9505
Epoch 778/1000
235/235 [==============================] - 0s 125us/step - loss: 0.0988 - accuracy: 0.9617 - val_loss: 0.1482 - val_accuracy: 0.9406
Epoch 779/1000
235/235 [==============================] - 0s 95us/step - loss: 0.0987 - accuracy: 0.9617 - val_loss: 0.1456 - va

235/235 [==============================] - 0s 195us/step - loss: 0.0981 - accuracy: 0.9617 - val_loss: 0.1443 - val_accuracy: 0.9505
Epoch 829/1000
235/235 [==============================] - 0s 203us/step - loss: 0.0980 - accuracy: 0.9617 - val_loss: 0.1443 - val_accuracy: 0.9505
Epoch 830/1000
235/235 [==============================] - 0s 259us/step - loss: 0.0982 - accuracy: 0.9617 - val_loss: 0.1453 - val_accuracy: 0.9505
Epoch 831/1000
235/235 [==============================] - 0s 213us/step - loss: 0.0979 - accuracy: 0.9617 - val_loss: 0.1471 - val_accuracy: 0.9406
Epoch 832/1000
235/235 [==============================] - 0s 137us/step - loss: 0.0981 - accuracy: 0.9617 - val_loss: 0.1448 - val_accuracy: 0.9505
Epoch 833/1000
235/235 [==============================] - 0s 153us/step - loss: 0.0979 - accuracy: 0.9617 - val_loss: 0.1450 - val_accuracy: 0.9505
Epoch 834/1000
235/235 [==============================] - 0s 151us/step - loss: 0.0982 - accuracy: 0.9617 - val_loss: 0.1449 - 

235/235 [==============================] - 0s 128us/step - loss: 0.0975 - accuracy: 0.9617 - val_loss: 0.1475 - val_accuracy: 0.9406
Epoch 884/1000
235/235 [==============================] - 0s 105us/step - loss: 0.0976 - accuracy: 0.9617 - val_loss: 0.1443 - val_accuracy: 0.9505
Epoch 885/1000
235/235 [==============================] - 0s 91us/step - loss: 0.0975 - accuracy: 0.9617 - val_loss: 0.1441 - val_accuracy: 0.9505
Epoch 886/1000
235/235 [==============================] - 0s 102us/step - loss: 0.0975 - accuracy: 0.9617 - val_loss: 0.1434 - val_accuracy: 0.9505
Epoch 887/1000
235/235 [==============================] - 0s 95us/step - loss: 0.0977 - accuracy: 0.9617 - val_loss: 0.1440 - val_accuracy: 0.9505
Epoch 888/1000
235/235 [==============================] - 0s 97us/step - loss: 0.0974 - accuracy: 0.9617 - val_loss: 0.1443 - val_accuracy: 0.9505
Epoch 889/1000
235/235 [==============================] - 0s 93us/step - loss: 0.0974 - accuracy: 0.9617 - val_loss: 0.1445 - val_

235/235 [==============================] - 0s 118us/step - loss: 0.0970 - accuracy: 0.9617 - val_loss: 0.1431 - val_accuracy: 0.9505
Epoch 939/1000
235/235 [==============================] - 0s 144us/step - loss: 0.0969 - accuracy: 0.9617 - val_loss: 0.1431 - val_accuracy: 0.9505
Epoch 940/1000
235/235 [==============================] - 0s 326us/step - loss: 0.0970 - accuracy: 0.9617 - val_loss: 0.1431 - val_accuracy: 0.9505
Epoch 941/1000
235/235 [==============================] - 0s 108us/step - loss: 0.0970 - accuracy: 0.9617 - val_loss: 0.1431 - val_accuracy: 0.9505
Epoch 942/1000
235/235 [==============================] - 0s 107us/step - loss: 0.0970 - accuracy: 0.9617 - val_loss: 0.1432 - val_accuracy: 0.9505
Epoch 943/1000
235/235 [==============================] - 0s 99us/step - loss: 0.0970 - accuracy: 0.9617 - val_loss: 0.1429 - val_accuracy: 0.9505
Epoch 944/1000
235/235 [==============================] - 0s 97us/step - loss: 0.0970 - accuracy: 0.9617 - val_loss: 0.1429 - va

235/235 [==============================] - 0s 91us/step - loss: 0.0966 - accuracy: 0.9617 - val_loss: 0.1425 - val_accuracy: 0.9505
Epoch 994/1000
235/235 [==============================] - 0s 98us/step - loss: 0.0966 - accuracy: 0.9617 - val_loss: 0.1423 - val_accuracy: 0.9505
Epoch 995/1000
235/235 [==============================] - 0s 141us/step - loss: 0.0966 - accuracy: 0.9617 - val_loss: 0.1423 - val_accuracy: 0.9505
Epoch 996/1000
235/235 [==============================] - 0s 170us/step - loss: 0.0967 - accuracy: 0.9617 - val_loss: 0.1426 - val_accuracy: 0.9505
Epoch 997/1000
235/235 [==============================] - 0s 200us/step - loss: 0.0967 - accuracy: 0.9617 - val_loss: 0.1414 - val_accuracy: 0.9505
Epoch 998/1000
235/235 [==============================] - 0s 173us/step - loss: 0.0967 - accuracy: 0.9617 - val_loss: 0.1420 - val_accuracy: 0.9505
Epoch 999/1000
235/235 [==============================] - 0s 203us/step - loss: 0.0966 - accuracy: 0.9617 - val_loss: 0.1425 - va

In [24]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

101/101 [==============================] - 0s 90us/step
combination test accuracy: 95.05%


In [25]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [26]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 235 samples, validate on 101 samples
Epoch 1/1000
235/235 [==============================] - 0s 1ms/step - loss: 2.9907 - accuracy: 0.4511 - val_loss: 0.7644 - val_accuracy: 0.3861
Epoch 2/1000
235/235 [==============================] - 0s 85us/step - loss: 2.2970 - accuracy: 0.4553 - val_loss: 0.7023 - val_accuracy: 0.6931
Epoch 3/1000
235/235 [==============================] - 0s 97us/step - loss: 1.6030 - accuracy: 0.5064 - val_loss: 0.6885 - val_accuracy: 0.6634
Epoch 4/1000
235/235 [==============================] - 0s 98us/step - loss: 1.8484 - accuracy: 0.4979 - val_loss: 0.6932 - val_accuracy: 0.6337
Epoch 5/1000
235/235 [==============================] - 0s 99us/step - loss: 2.0189 - accuracy: 0.5277 - val_loss: 0.6804 - val_accuracy: 0.6337
Epoch 6/1000
235/235 [==============================] - 0s 96us/step - loss: 1.9842 - accuracy: 0.5574 - val_loss: 0.6730 - val_accuracy: 0.6634
Epoch 7/1000
235/235 [==============================] - 0s 104us/step - loss: 2.0426 

235/235 [==============================] - 0s 155us/step - loss: 2.5641 - accuracy: 0.7872 - val_loss: 0.4833 - val_accuracy: 0.6634
Epoch 57/1000
235/235 [==============================] - 0s 196us/step - loss: 1.8894 - accuracy: 0.8468 - val_loss: 0.4868 - val_accuracy: 0.6535
Epoch 58/1000
235/235 [==============================] - 0s 247us/step - loss: 2.0934 - accuracy: 0.8213 - val_loss: 0.4890 - val_accuracy: 0.6535
Epoch 59/1000
235/235 [==============================] - 0s 147us/step - loss: 1.4383 - accuracy: 0.8809 - val_loss: 0.4834 - val_accuracy: 0.6535
Epoch 60/1000
235/235 [==============================] - 0s 118us/step - loss: 1.6399 - accuracy: 0.8553 - val_loss: 0.4809 - val_accuracy: 0.6634
Epoch 61/1000
235/235 [==============================] - 0s 128us/step - loss: 1.8060 - accuracy: 0.8596 - val_loss: 0.4801 - val_accuracy: 0.8911
Epoch 62/1000
235/235 [==============================] - 0s 206us/step - loss: 1.6933 - accuracy: 0.8638 - val_loss: 0.4740 - val_ac

Epoch 112/1000
235/235 [==============================] - 0s 106us/step - loss: 1.7473 - accuracy: 0.8511 - val_loss: 0.3809 - val_accuracy: 0.9208
Epoch 113/1000
235/235 [==============================] - 0s 111us/step - loss: 2.1225 - accuracy: 0.8340 - val_loss: 0.3771 - val_accuracy: 0.9208
Epoch 114/1000
235/235 [==============================] - 0s 111us/step - loss: 1.7364 - accuracy: 0.8511 - val_loss: 0.3696 - val_accuracy: 0.9307
Epoch 115/1000
235/235 [==============================] - 0s 100us/step - loss: 1.7515 - accuracy: 0.8468 - val_loss: 0.3734 - val_accuracy: 0.9208
Epoch 116/1000
235/235 [==============================] - 0s 93us/step - loss: 1.2902 - accuracy: 0.8766 - val_loss: 0.3737 - val_accuracy: 0.9208
Epoch 117/1000
235/235 [==============================] - 0s 103us/step - loss: 1.3486 - accuracy: 0.8723 - val_loss: 0.3773 - val_accuracy: 0.9208
Epoch 118/1000
235/235 [==============================] - 0s 103us/step - loss: 1.4230 - accuracy: 0.8681 - val_l

235/235 [==============================] - 0s 162us/step - loss: 1.9428 - accuracy: 0.8553 - val_loss: 0.3402 - val_accuracy: 0.9208
Epoch 168/1000
235/235 [==============================] - 0s 120us/step - loss: 1.7766 - accuracy: 0.8468 - val_loss: 0.3320 - val_accuracy: 0.9307
Epoch 169/1000
235/235 [==============================] - 0s 106us/step - loss: 1.5797 - accuracy: 0.8596 - val_loss: 0.3706 - val_accuracy: 0.9109
Epoch 170/1000
235/235 [==============================] - 0s 89us/step - loss: 1.7247 - accuracy: 0.8511 - val_loss: 0.3348 - val_accuracy: 0.9208
Epoch 171/1000
235/235 [==============================] - 0s 90us/step - loss: 1.6322 - accuracy: 0.8681 - val_loss: 0.3407 - val_accuracy: 0.9109
Epoch 172/1000
235/235 [==============================] - 0s 91us/step - loss: 1.5133 - accuracy: 0.8596 - val_loss: 0.3346 - val_accuracy: 0.9208
Epoch 173/1000
235/235 [==============================] - 0s 86us/step - loss: 1.5120 - accuracy: 0.8681 - val_loss: 0.3471 - val_

Epoch 223/1000
235/235 [==============================] - 0s 92us/step - loss: 1.9493 - accuracy: 0.8383 - val_loss: 0.3180 - val_accuracy: 0.9208
Epoch 224/1000
235/235 [==============================] - 0s 89us/step - loss: 1.7614 - accuracy: 0.8426 - val_loss: 0.3200 - val_accuracy: 0.9109
Epoch 225/1000
235/235 [==============================] - 0s 95us/step - loss: 1.7595 - accuracy: 0.8468 - val_loss: 0.3166 - val_accuracy: 0.9208
Epoch 226/1000
235/235 [==============================] - 0s 92us/step - loss: 1.4985 - accuracy: 0.8638 - val_loss: 0.3167 - val_accuracy: 0.9208
Epoch 227/1000
235/235 [==============================] - 0s 92us/step - loss: 1.6944 - accuracy: 0.8511 - val_loss: 0.3204 - val_accuracy: 0.9109
Epoch 228/1000
235/235 [==============================] - 0s 91us/step - loss: 1.7465 - accuracy: 0.8511 - val_loss: 0.3137 - val_accuracy: 0.9208
Epoch 229/1000
235/235 [==============================] - 0s 92us/step - loss: 1.6967 - accuracy: 0.8511 - val_loss: 0

...

235/235 [==============================] - 0s 100us/step - loss: 1.9515 - accuracy: 0.8340 - val_loss: 0.3191 - val_accuracy: 0.9109
Epoch 776/1000
235/235 [==============================] - 0s 94us/step - loss: 1.4044 - accuracy: 0.8851 - val_loss: 0.3028 - val_accuracy: 0.9208
Epoch 777/1000
235/235 [==============================] - 0s 95us/step - loss: 1.5492 - accuracy: 0.8681 - val_loss: 0.3090 - val_accuracy: 0.9109
Epoch 778/1000
235/235 [==============================] - 0s 94us/step - loss: 1.4958 - accuracy: 0.8638 - val_loss: 0.3053 - val_accuracy: 0.9109
Epoch 779/1000
235/235 [==============================] - 0s 95us/step - loss: 2.2120 - accuracy: 0.8170 - val_loss: 0.3059 - val_accuracy: 0.9109
Epoch 780/1000
235/235 [==============================] - 0s 91us/step - loss: 1.4194 - accuracy: 0.8766 - val_loss: 0.3126 - val_accuracy: 0.9109
Epoch 781/1000
235/235 [==============================] - 0s 94us/step - loss: 1.7463 - accuracy: 0.8511 - val_loss: 0.3015 - val_ac

Epoch 831/1000
235/235 [==============================] - 0s 98us/step - loss: 1.6158 - accuracy: 0.8596 - val_loss: 0.3008 - val_accuracy: 0.9109
Epoch 832/1000
235/235 [==============================] - 0s 98us/step - loss: 1.3595 - accuracy: 0.8723 - val_loss: 0.2972 - val_accuracy: 0.9109
Epoch 833/1000
235/235 [==============================] - 0s 94us/step - loss: 1.2960 - accuracy: 0.8766 - val_loss: 0.2982 - val_accuracy: 0.9109
Epoch 834/1000
235/235 [==============================] - 0s 98us/step - loss: 1.5577 - accuracy: 0.8596 - val_loss: 0.3004 - val_accuracy: 0.9109
Epoch 835/1000
235/235 [==============================] - 0s 94us/step - loss: 1.4183 - accuracy: 0.8723 - val_loss: 0.2992 - val_accuracy: 0.9109
Epoch 836/1000
235/235 [==============================] - 0s 94us/step - loss: 1.6949 - accuracy: 0.8468 - val_loss: 0.2974 - val_accuracy: 0.9109
Epoch 837/1000
235/235 [==============================] - ETA: 0s - loss: 2.5361 - accuracy: 0.81 - 0s 102us/step - lo

235/235 [==============================] - 0s 107us/step - loss: 1.9093 - accuracy: 0.8468 - val_loss: 0.2870 - val_accuracy: 0.9109
Epoch 887/1000
235/235 [==============================] - 0s 115us/step - loss: 1.6604 - accuracy: 0.8596 - val_loss: 0.2928 - val_accuracy: 0.9109
Epoch 888/1000
235/235 [==============================] - 0s 114us/step - loss: 1.2201 - accuracy: 0.8766 - val_loss: 0.2852 - val_accuracy: 0.9109
Epoch 889/1000
235/235 [==============================] - 0s 111us/step - loss: 1.4619 - accuracy: 0.8723 - val_loss: 0.2856 - val_accuracy: 0.9109
Epoch 890/1000
235/235 [==============================] - 0s 110us/step - loss: 1.6708 - accuracy: 0.8511 - val_loss: 0.2915 - val_accuracy: 0.9109
Epoch 891/1000
235/235 [==============================] - 0s 110us/step - loss: 2.1201 - accuracy: 0.8255 - val_loss: 0.2861 - val_accuracy: 0.9109
Epoch 892/1000
235/235 [==============================] - 0s 117us/step - loss: 1.9988 - accuracy: 0.8298 - val_loss: 0.2896 - 

235/235 [==============================] - 0s 94us/step - loss: 1.7297 - accuracy: 0.8511 - val_loss: 0.2856 - val_accuracy: 0.9109
Epoch 942/1000
235/235 [==============================] - 0s 97us/step - loss: 1.4008 - accuracy: 0.8723 - val_loss: 0.2922 - val_accuracy: 0.9109
Epoch 943/1000
235/235 [==============================] - 0s 104us/step - loss: 2.0549 - accuracy: 0.8298 - val_loss: 0.2871 - val_accuracy: 0.9109
Epoch 944/1000
235/235 [==============================] - 0s 105us/step - loss: 1.3523 - accuracy: 0.8681 - val_loss: 0.2955 - val_accuracy: 0.9010
Epoch 945/1000
235/235 [==============================] - 0s 95us/step - loss: 1.5722 - accuracy: 0.8766 - val_loss: 0.2878 - val_accuracy: 0.9109
Epoch 946/1000
235/235 [==============================] - 0s 112us/step - loss: 1.2038 - accuracy: 0.8851 - val_loss: 0.2919 - val_accuracy: 0.9109
Epoch 947/1000
235/235 [==============================] - 0s 112us/step - loss: 1.8647 - accuracy: 0.8383 - val_loss: 0.2862 - val

Epoch 997/1000
235/235 [==============================] - 0s 109us/step - loss: 1.3445 - accuracy: 0.8723 - val_loss: 0.2792 - val_accuracy: 0.9208
Epoch 998/1000
235/235 [==============================] - 0s 102us/step - loss: 2.2551 - accuracy: 0.8128 - val_loss: 0.2806 - val_accuracy: 0.9208
Epoch 999/1000
235/235 [==============================] - 0s 86us/step - loss: 1.1322 - accuracy: 0.8936 - val_loss: 0.2864 - val_accuracy: 0.9109
Epoch 1000/1000
235/235 [==============================] - 0s 94us/step - loss: 1.6591 - accuracy: 0.8596 - val_loss: 0.2862 - val_accuracy: 0.9109


In [28]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

101/101 [==============================] - 0s 67us/step
combination test accuracy: 91.09%


In [29]:
#### improve neural network
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [30]:
model1_comb3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 235 samples, validate on 101 samples
Epoch 1/1000
235/235 [==============================] - 0s 730us/step - loss: 2.9856 - accuracy: 0.4936 - val_loss: 0.7379 - val_accuracy: 0.5842
Epoch 2/1000
235/235 [==============================] - 0s 89us/step - loss: 2.2260 - accuracy: 0.5064 - val_loss: 0.7051 - val_accuracy: 0.6931
Epoch 3/1000
235/235 [==============================] - 0s 108us/step - loss: 1.7082 - accuracy: 0.5149 - val_loss: 0.6933 - val_accuracy: 0.5842
Epoch 4/1000
235/235 [==============================] - 0s 101us/step - loss: 2.0093 - accuracy: 0.5021 - val_loss: 0.6843 - val_accuracy: 0.5941
Epoch 5/1000
235/235 [==============================] - 0s 102us/step - loss: 2.0016 - accuracy: 0.4936 - val_loss: 0.6757 - val_accuracy: 0.6832
Epoch 6/1000
235/235 [==============================] - 0s 104us/step - loss: 1.8444 - accuracy: 0.4596 - val_loss: 0.6691 - val_accuracy: 0.6931
Epoch 7/1000
235/235 [==============================] - 0s 102us/step - loss: 2

235/235 [==============================] - 0s 104us/step - loss: 1.6363 - accuracy: 0.8511 - val_loss: 0.4863 - val_accuracy: 0.5941
Epoch 57/1000
235/235 [==============================] - 0s 98us/step - loss: 1.6942 - accuracy: 0.8681 - val_loss: 0.4270 - val_accuracy: 0.7228
Epoch 58/1000
235/235 [==============================] - 0s 90us/step - loss: 1.3789 - accuracy: 0.8638 - val_loss: 0.4232 - val_accuracy: 0.7228
Epoch 59/1000
235/235 [==============================] - 0s 96us/step - loss: 1.4277 - accuracy: 0.8638 - val_loss: 0.4181 - val_accuracy: 0.7228
Epoch 60/1000
235/235 [==============================] - 0s 102us/step - loss: 1.5525 - accuracy: 0.8596 - val_loss: 0.4135 - val_accuracy: 0.7327
Epoch 61/1000
235/235 [==============================] - 0s 96us/step - loss: 1.6365 - accuracy: 0.8553 - val_loss: 0.4239 - val_accuracy: 0.7228
Epoch 62/1000
235/235 [==============================] - 0s 111us/step - loss: 1.8327 - accuracy: 0.8340 - val_loss: 0.4214 - val_accura

Epoch 112/1000
235/235 [==============================] - 0s 96us/step - loss: 1.2710 - accuracy: 0.8809 - val_loss: 0.3210 - val_accuracy: 0.9109
Epoch 113/1000
235/235 [==============================] - 0s 96us/step - loss: 1.5871 - accuracy: 0.8596 - val_loss: 0.3415 - val_accuracy: 0.9208
Epoch 114/1000
235/235 [==============================] - 0s 99us/step - loss: 1.1403 - accuracy: 0.8894 - val_loss: 0.3388 - val_accuracy: 0.9208
Epoch 115/1000
235/235 [==============================] - 0s 108us/step - loss: 1.5350 - accuracy: 0.8596 - val_loss: 0.3345 - val_accuracy: 0.9109
Epoch 116/1000
235/235 [==============================] - 0s 91us/step - loss: 1.4587 - accuracy: 0.8723 - val_loss: 0.3375 - val_accuracy: 0.9109
Epoch 117/1000
235/235 [==============================] - 0s 103us/step - loss: 1.9770 - accuracy: 0.8383 - val_loss: 0.3407 - val_accuracy: 0.9109
Epoch 118/1000
235/235 [==============================] - 0s 103us/step - loss: 1.7039 - accuracy: 0.8596 - val_loss

235/235 [==============================] - 0s 100us/step - loss: 1.4194 - accuracy: 0.8723 - val_loss: 0.2903 - val_accuracy: 0.9208
Epoch 168/1000
235/235 [==============================] - 0s 99us/step - loss: 1.7238 - accuracy: 0.8638 - val_loss: 0.3036 - val_accuracy: 0.9208
Epoch 169/1000
235/235 [==============================] - 0s 96us/step - loss: 2.0229 - accuracy: 0.8255 - val_loss: 0.3007 - val_accuracy: 0.9208
Epoch 170/1000
235/235 [==============================] - 0s 90us/step - loss: 1.8318 - accuracy: 0.8426 - val_loss: 0.3300 - val_accuracy: 0.9208
Epoch 171/1000
235/235 [==============================] - 0s 98us/step - loss: 1.4856 - accuracy: 0.8681 - val_loss: 0.2902 - val_accuracy: 0.9208
Epoch 172/1000
235/235 [==============================] - ETA: 0s - loss: 3.0194 - accuracy: 0.78 - 0s 97us/step - loss: 1.8167 - accuracy: 0.8468 - val_loss: 0.2889 - val_accuracy: 0.9208
Epoch 173/1000
235/235 [==============================] - 0s 112us/step - loss: 1.6025 - a

235/235 [==============================] - 0s 100us/step - loss: 1.2053 - accuracy: 0.8979 - val_loss: 0.2962 - val_accuracy: 0.9307
Epoch 223/1000
235/235 [==============================] - 0s 105us/step - loss: 2.1311 - accuracy: 0.8340 - val_loss: 0.3150 - val_accuracy: 0.9208
Epoch 224/1000
235/235 [==============================] - 0s 101us/step - loss: 1.5100 - accuracy: 0.8596 - val_loss: 0.3188 - val_accuracy: 0.9208
Epoch 225/1000
235/235 [==============================] - 0s 98us/step - loss: 1.4980 - accuracy: 0.8638 - val_loss: 0.2960 - val_accuracy: 0.9208
Epoch 226/1000
235/235 [==============================] - ETA: 0s - loss: 3.4542 - accuracy: 0.78 - 0s 101us/step - loss: 1.6705 - accuracy: 0.8638 - val_loss: 0.2912 - val_accuracy: 0.9307
Epoch 227/1000
235/235 [==============================] - 0s 105us/step - loss: 1.5462 - accuracy: 0.8681 - val_loss: 0.2942 - val_accuracy: 0.9307
Epoch 228/1000
235/235 [==============================] - 0s 102us/step - loss: 1.7608

...

235/235 [==============================] - 0s 135us/step - loss: 1.7455 - accuracy: 0.8511 - val_loss: 0.2829 - val_accuracy: 0.9208
Epoch 773/1000
235/235 [==============================] - 0s 137us/step - loss: 1.4283 - accuracy: 0.8681 - val_loss: 0.2818 - val_accuracy: 0.9208
Epoch 774/1000
235/235 [==============================] - 0s 127us/step - loss: 1.3542 - accuracy: 0.8766 - val_loss: 0.2851 - val_accuracy: 0.9208
Epoch 775/1000
235/235 [==============================] - 0s 126us/step - loss: 1.8932 - accuracy: 0.8340 - val_loss: 0.2848 - val_accuracy: 0.9208
Epoch 776/1000
235/235 [==============================] - 0s 138us/step - loss: 1.2241 - accuracy: 0.8851 - val_loss: 0.2827 - val_accuracy: 0.9208
Epoch 777/1000
235/235 [==============================] - 0s 116us/step - loss: 1.5960 - accuracy: 0.8681 - val_loss: 0.2825 - val_accuracy: 0.9208
Epoch 778/1000
235/235 [==============================] - 0s 134us/step - loss: 1.5675 - accuracy: 0.8553 - val_loss: 0.2818 - 

235/235 [==============================] - 0s 105us/step - loss: 1.9486 - accuracy: 0.8340 - val_loss: 0.2787 - val_accuracy: 0.9208
Epoch 828/1000
235/235 [==============================] - 0s 95us/step - loss: 1.8838 - accuracy: 0.8383 - val_loss: 0.2809 - val_accuracy: 0.9208
Epoch 829/1000
235/235 [==============================] - 0s 97us/step - loss: 1.4840 - accuracy: 0.8681 - val_loss: 0.2811 - val_accuracy: 0.9208
Epoch 830/1000
235/235 [==============================] - 0s 97us/step - loss: 1.7441 - accuracy: 0.8511 - val_loss: 0.2787 - val_accuracy: 0.9208
Epoch 831/1000
235/235 [==============================] - 0s 118us/step - loss: 1.4092 - accuracy: 0.8766 - val_loss: 0.2770 - val_accuracy: 0.9208
Epoch 832/1000
235/235 [==============================] - 0s 103us/step - loss: 1.6238 - accuracy: 0.8553 - val_loss: 0.2783 - val_accuracy: 0.9208
Epoch 833/1000
235/235 [==============================] - 0s 96us/step - loss: 1.7624 - accuracy: 0.8681 - val_loss: 0.2747 - val_

235/235 [==============================] - 0s 101us/step - loss: 2.1433 - accuracy: 0.8213 - val_loss: 0.2787 - val_accuracy: 0.9208
Epoch 883/1000
235/235 [==============================] - 0s 99us/step - loss: 1.9479 - accuracy: 0.8340 - val_loss: 0.2804 - val_accuracy: 0.9208
Epoch 884/1000
235/235 [==============================] - 0s 92us/step - loss: 1.8083 - accuracy: 0.8468 - val_loss: 0.2807 - val_accuracy: 0.9208
Epoch 885/1000
235/235 [==============================] - 0s 94us/step - loss: 1.5395 - accuracy: 0.8681 - val_loss: 0.2784 - val_accuracy: 0.9208
Epoch 886/1000
235/235 [==============================] - 0s 97us/step - loss: 1.4738 - accuracy: 0.8723 - val_loss: 0.2796 - val_accuracy: 0.9208
Epoch 887/1000
235/235 [==============================] - 0s 108us/step - loss: 1.4824 - accuracy: 0.8681 - val_loss: 0.2786 - val_accuracy: 0.9208
Epoch 888/1000
235/235 [==============================] - 0s 111us/step - loss: 1.1012 - accuracy: 0.8894 - val_loss: 0.2771 - val_

235/235 [==============================] - 0s 115us/step - loss: 1.5473 - accuracy: 0.8638 - val_loss: 0.2752 - val_accuracy: 0.9208
Epoch 938/1000
235/235 [==============================] - 0s 106us/step - loss: 2.0874 - accuracy: 0.8213 - val_loss: 0.2781 - val_accuracy: 0.9208
Epoch 939/1000
235/235 [==============================] - 0s 96us/step - loss: 1.7612 - accuracy: 0.8426 - val_loss: 0.2792 - val_accuracy: 0.9208
Epoch 940/1000
235/235 [==============================] - 0s 102us/step - loss: 2.0220 - accuracy: 0.8255 - val_loss: 0.2793 - val_accuracy: 0.9208
Epoch 941/1000
235/235 [==============================] - 0s 103us/step - loss: 1.7154 - accuracy: 0.8638 - val_loss: 0.2801 - val_accuracy: 0.9208
Epoch 942/1000
235/235 [==============================] - 0s 129us/step - loss: 1.6222 - accuracy: 0.8553 - val_loss: 0.2799 - val_accuracy: 0.9208
Epoch 943/1000
235/235 [==============================] - 0s 134us/step - loss: 1.4359 - accuracy: 0.8638 - val_loss: 0.2809 - v

235/235 [==============================] - 0s 100us/step - loss: 1.6683 - accuracy: 0.8596 - val_loss: 0.2771 - val_accuracy: 0.9208
Epoch 993/1000
235/235 [==============================] - 0s 99us/step - loss: 1.3885 - accuracy: 0.8851 - val_loss: 0.2742 - val_accuracy: 0.9208
Epoch 994/1000
235/235 [==============================] - 0s 93us/step - loss: 1.5739 - accuracy: 0.8766 - val_loss: 0.2731 - val_accuracy: 0.9208
Epoch 995/1000
235/235 [==============================] - 0s 95us/step - loss: 1.4829 - accuracy: 0.8681 - val_loss: 0.2732 - val_accuracy: 0.9208
Epoch 996/1000
235/235 [==============================] - 0s 101us/step - loss: 2.3573 - accuracy: 0.8000 - val_loss: 0.2751 - val_accuracy: 0.9208
Epoch 997/1000
235/235 [==============================] - 0s 97us/step - loss: 1.7520 - accuracy: 0.8468 - val_loss: 0.2747 - val_accuracy: 0.9208
Epoch 998/1000
235/235 [==============================] - 0s 110us/step - loss: 1.6958 - accuracy: 0.8468 - val_loss: 0.2746 - val_

In [32]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

101/101 [==============================] - 0s 89us/step
combination test accuracy: 92.08%


In [33]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [34]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/1000
305/305 [==============================] - 0s 345us/step - loss: 21.6049 - accuracy: 0.5541 - val_loss: 0.6406 - val_accuracy: 0.6794
Epoch 2/1000
305/305 [==============================] - 0s 77us/step - loss: 1.5872 - accuracy: 0.6000 - val_loss: 1.0334 - val_accuracy: 0.5344
Epoch 3/1000
305/305 [==============================] - 0s 83us/step - loss: 0.9549 - accuracy: 0.6066 - val_loss: 0.6422 - val_accuracy: 0.7099
Epoch 4/1000
305/305 [==============================] - 0s 81us/step - loss: 0.6406 - accuracy: 0.7344 - val_loss: 0.6145 - val_accuracy: 0.7405
Epoch 5/1000
305/305 [==============================] - 0s 86us/step - loss: 0.6271 - accuracy: 0.7115 - val_loss: 0.6032 - val_accuracy: 0.7786
Epoch 6/1000
305/305 [==============================] - 0s 83us/step - loss: 0.6144 - accuracy: 0.7246 - val_loss: 0.5891 - val_accuracy: 0.7634
Epoch 7/1000
305/305 [==============================] - 0s 82us/step - loss: 0.601

Epoch 57/1000
305/305 [==============================] - 0s 80us/step - loss: 0.3824 - accuracy: 0.8295 - val_loss: 0.3952 - val_accuracy: 0.8092
Epoch 58/1000
305/305 [==============================] - 0s 74us/step - loss: 0.3840 - accuracy: 0.8361 - val_loss: 0.3983 - val_accuracy: 0.8092
Epoch 59/1000
305/305 [==============================] - 0s 77us/step - loss: 0.3812 - accuracy: 0.8361 - val_loss: 0.3982 - val_accuracy: 0.8092
Epoch 60/1000
305/305 [==============================] - 0s 81us/step - loss: 0.3794 - accuracy: 0.8295 - val_loss: 0.3965 - val_accuracy: 0.8168
Epoch 61/1000
305/305 [==============================] - 0s 78us/step - loss: 0.3772 - accuracy: 0.8361 - val_loss: 0.3878 - val_accuracy: 0.8321
Epoch 62/1000
305/305 [==============================] - 0s 73us/step - loss: 0.3772 - accuracy: 0.8361 - val_loss: 0.3915 - val_accuracy: 0.8168
Epoch 63/1000
305/305 [==============================] - 0s 78us/step - loss: 0.3765 - accuracy: 0.8361 - val_loss: 0.3858 -

305/305 [==============================] - 0s 88us/step - loss: 0.3344 - accuracy: 0.8459 - val_loss: 0.3469 - val_accuracy: 0.8244
Epoch 114/1000
305/305 [==============================] - 0s 82us/step - loss: 0.3337 - accuracy: 0.8492 - val_loss: 0.3556 - val_accuracy: 0.8244
Epoch 115/1000
305/305 [==============================] - 0s 82us/step - loss: 0.3327 - accuracy: 0.8459 - val_loss: 0.3594 - val_accuracy: 0.8244
Epoch 116/1000
305/305 [==============================] - 0s 97us/step - loss: 0.3334 - accuracy: 0.8492 - val_loss: 0.3698 - val_accuracy: 0.8168
Epoch 117/1000
305/305 [==============================] - 0s 89us/step - loss: 0.3315 - accuracy: 0.8492 - val_loss: 0.3504 - val_accuracy: 0.8244
Epoch 118/1000
305/305 [==============================] - 0s 86us/step - loss: 0.3315 - accuracy: 0.8459 - val_loss: 0.3440 - val_accuracy: 0.8244
Epoch 119/1000
305/305 [==============================] - 0s 132us/step - loss: 0.3334 - accuracy: 0.8492 - val_loss: 0.3511 - val_ac

Epoch 169/1000
305/305 [==============================] - 0s 81us/step - loss: 0.3105 - accuracy: 0.8492 - val_loss: 0.3587 - val_accuracy: 0.8168
Epoch 170/1000
305/305 [==============================] - 0s 77us/step - loss: 0.3128 - accuracy: 0.8492 - val_loss: 0.3297 - val_accuracy: 0.8244
Epoch 171/1000
305/305 [==============================] - 0s 79us/step - loss: 0.3119 - accuracy: 0.8492 - val_loss: 0.3447 - val_accuracy: 0.8244
Epoch 172/1000
305/305 [==============================] - 0s 82us/step - loss: 0.3105 - accuracy: 0.8492 - val_loss: 0.3528 - val_accuracy: 0.8168
Epoch 173/1000
305/305 [==============================] - 0s 80us/step - loss: 0.3104 - accuracy: 0.8492 - val_loss: 0.3513 - val_accuracy: 0.8244
Epoch 174/1000
305/305 [==============================] - 0s 75us/step - loss: 0.3108 - accuracy: 0.8492 - val_loss: 0.3344 - val_accuracy: 0.8244
Epoch 175/1000
305/305 [==============================] - 0s 78us/step - loss: 0.3087 - accuracy: 0.8492 - val_loss: 0

Epoch 225/1000
305/305 [==============================] - 0s 81us/step - loss: 0.2945 - accuracy: 0.8525 - val_loss: 0.3211 - val_accuracy: 0.8321
Epoch 226/1000
305/305 [==============================] - 0s 77us/step - loss: 0.2964 - accuracy: 0.8525 - val_loss: 0.3309 - val_accuracy: 0.8244
Epoch 227/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2937 - accuracy: 0.8525 - val_loss: 0.3317 - val_accuracy: 0.8244
Epoch 228/1000
305/305 [==============================] - 0s 76us/step - loss: 0.2979 - accuracy: 0.8492 - val_loss: 0.3197 - val_accuracy: 0.8244
Epoch 229/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2969 - accuracy: 0.8492 - val_loss: 0.3221 - val_accuracy: 0.8244
Epoch 230/1000
305/305 [==============================] - 0s 74us/step - loss: 0.2923 - accuracy: 0.8492 - val_loss: 0.3154 - val_accuracy: 0.8244
Epoch 231/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2926 - accuracy: 0.8525 - val_loss: 0

...

Epoch 728/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2328 - accuracy: 0.8787 - val_loss: 0.2967 - val_accuracy: 0.8473
Epoch 729/1000
305/305 [==============================] - 0s 76us/step - loss: 0.2341 - accuracy: 0.8787 - val_loss: 0.2808 - val_accuracy: 0.8473
Epoch 730/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2355 - accuracy: 0.8787 - val_loss: 0.3289 - val_accuracy: 0.8321
Epoch 731/1000
305/305 [==============================] - 0s 76us/step - loss: 0.2362 - accuracy: 0.8787 - val_loss: 0.2784 - val_accuracy: 0.8473
Epoch 732/1000
305/305 [==============================] - 0s 75us/step - loss: 0.2348 - accuracy: 0.8787 - val_loss: 0.2972 - val_accuracy: 0.8397
Epoch 733/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2325 - accuracy: 0.8820 - val_loss: 0.2806 - val_accuracy: 0.8473
Epoch 734/1000
305/305 [==============================] - 0s 85us/step - loss: 0.2368 - accuracy: 0.8787 - val_loss: 0

Epoch 784/1000
305/305 [==============================] - 0s 84us/step - loss: 0.2314 - accuracy: 0.8787 - val_loss: 0.2753 - val_accuracy: 0.8550
Epoch 785/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2346 - accuracy: 0.8787 - val_loss: 0.2814 - val_accuracy: 0.8473
Epoch 786/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2356 - accuracy: 0.8787 - val_loss: 0.2864 - val_accuracy: 0.8473
Epoch 787/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2314 - accuracy: 0.8787 - val_loss: 0.2714 - val_accuracy: 0.8550
Epoch 788/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2367 - accuracy: 0.8787 - val_loss: 0.2872 - val_accuracy: 0.8473
Epoch 789/1000
305/305 [==============================] - 0s 77us/step - loss: 0.2316 - accuracy: 0.8754 - val_loss: 0.2738 - val_accuracy: 0.8550
Epoch 790/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2330 - accuracy: 0.8787 - val_loss: 0

Epoch 840/1000
305/305 [==============================] - 0s 81us/step - loss: 0.2350 - accuracy: 0.8787 - val_loss: 0.2861 - val_accuracy: 0.8473
Epoch 841/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2290 - accuracy: 0.8787 - val_loss: 0.2845 - val_accuracy: 0.8473
Epoch 842/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2318 - accuracy: 0.8787 - val_loss: 0.2874 - val_accuracy: 0.8473
Epoch 843/1000
305/305 [==============================] - 0s 74us/step - loss: 0.2300 - accuracy: 0.8820 - val_loss: 0.2854 - val_accuracy: 0.8473
Epoch 844/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2296 - accuracy: 0.8820 - val_loss: 0.2954 - val_accuracy: 0.8397
Epoch 845/1000
305/305 [==============================] - 0s 77us/step - loss: 0.2301 - accuracy: 0.8820 - val_loss: 0.2787 - val_accuracy: 0.8550
Epoch 846/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2306 - accuracy: 0.8787 - val_loss: 0

Epoch 896/1000
305/305 [==============================] - 0s 81us/step - loss: 0.2289 - accuracy: 0.8820 - val_loss: 0.2832 - val_accuracy: 0.8473
Epoch 897/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2298 - accuracy: 0.8820 - val_loss: 0.2749 - val_accuracy: 0.8550
Epoch 898/1000
305/305 [==============================] - 0s 75us/step - loss: 0.2289 - accuracy: 0.8820 - val_loss: 0.2873 - val_accuracy: 0.8397
Epoch 899/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2291 - accuracy: 0.8820 - val_loss: 0.3426 - val_accuracy: 0.8244
Epoch 900/1000
305/305 [==============================] - 0s 75us/step - loss: 0.2295 - accuracy: 0.8820 - val_loss: 0.2717 - val_accuracy: 0.8550
Epoch 901/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2290 - accuracy: 0.8820 - val_loss: 0.2844 - val_accuracy: 0.8473
Epoch 902/1000
305/305 [==============================] - 0s 73us/step - loss: 0.2290 - accuracy: 0.8820 - val_loss: 0

Epoch 952/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2274 - accuracy: 0.8885 - val_loss: 0.2684 - val_accuracy: 0.8550
Epoch 953/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2285 - accuracy: 0.8820 - val_loss: 0.2831 - val_accuracy: 0.8550
Epoch 954/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2280 - accuracy: 0.8820 - val_loss: 0.2936 - val_accuracy: 0.8473
Epoch 955/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2270 - accuracy: 0.8820 - val_loss: 0.2956 - val_accuracy: 0.8473
Epoch 956/1000
305/305 [==============================] - 0s 81us/step - loss: 0.2267 - accuracy: 0.8820 - val_loss: 0.2774 - val_accuracy: 0.8550
Epoch 957/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2269 - accuracy: 0.8820 - val_loss: 0.2813 - val_accuracy: 0.8550
Epoch 958/1000
305/305 [==============================] - 0s 76us/step - loss: 0.2288 - accuracy: 0.8787 - val_loss: 0

In [36]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

131/131 [==============================] - 0s 78us/step
over-sampling test accuracy: 82.44%


In [37]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [38]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/1000
305/305 [==============================] - 0s 393us/step - loss: 4.9726 - accuracy: 0.4721 - val_loss: 0.6539 - val_accuracy: 0.6870
Epoch 2/1000
305/305 [==============================] - 0s 76us/step - loss: 0.6386 - accuracy: 0.7115 - val_loss: 0.5911 - val_accuracy: 0.7634
Epoch 3/1000
305/305 [==============================] - 0s 85us/step - loss: 0.6017 - accuracy: 0.7049 - val_loss: 0.5636 - val_accuracy: 0.7405
Epoch 4/1000
305/305 [==============================] - 0s 90us/step - loss: 0.5677 - accuracy: 0.7049 - val_loss: 0.5402 - val_accuracy: 0.7634
Epoch 5/1000
305/305 [==============================] - 0s 87us/step - loss: 0.5537 - accuracy: 0.7180 - val_loss: 0.5240 - val_accuracy: 0.7634
Epoch 6/1000
305/305 [==============================] - 0s 95us/step - loss: 0.5390 - accuracy: 0.7246 - val_loss: 0.5098 - val_accuracy: 0.7481
Epoch 7/1000
305/305 [==============================] - 0s 90us/step - loss: 0.5361

305/305 [==============================] - 0s 85us/step - loss: 0.3727 - accuracy: 0.8328 - val_loss: 0.3673 - val_accuracy: 0.8397
Epoch 57/1000
305/305 [==============================] - 0s 84us/step - loss: 0.3711 - accuracy: 0.8426 - val_loss: 0.3713 - val_accuracy: 0.8244
Epoch 58/1000
305/305 [==============================] - 0s 78us/step - loss: 0.3705 - accuracy: 0.8393 - val_loss: 0.4205 - val_accuracy: 0.8015
Epoch 59/1000
305/305 [==============================] - 0s 78us/step - loss: 0.3696 - accuracy: 0.8361 - val_loss: 0.3719 - val_accuracy: 0.8397
Epoch 60/1000
305/305 [==============================] - 0s 88us/step - loss: 0.3797 - accuracy: 0.8393 - val_loss: 0.3794 - val_accuracy: 0.8244
Epoch 61/1000
305/305 [==============================] - 0s 77us/step - loss: 0.3764 - accuracy: 0.8361 - val_loss: 0.4006 - val_accuracy: 0.8168
Epoch 62/1000
305/305 [==============================] - 0s 81us/step - loss: 0.3668 - accuracy: 0.8361 - val_loss: 0.4171 - val_accuracy:

305/305 [==============================] - 0s 88us/step - loss: 0.3316 - accuracy: 0.8393 - val_loss: 0.3495 - val_accuracy: 0.8244
Epoch 113/1000
305/305 [==============================] - 0s 84us/step - loss: 0.3221 - accuracy: 0.8492 - val_loss: 0.3349 - val_accuracy: 0.8244
Epoch 114/1000
305/305 [==============================] - 0s 79us/step - loss: 0.3282 - accuracy: 0.8426 - val_loss: 0.3315 - val_accuracy: 0.8244
Epoch 115/1000
305/305 [==============================] - 0s 79us/step - loss: 0.3254 - accuracy: 0.8492 - val_loss: 0.3520 - val_accuracy: 0.8321
Epoch 116/1000
305/305 [==============================] - 0s 85us/step - loss: 0.3281 - accuracy: 0.8426 - val_loss: 0.3577 - val_accuracy: 0.8244
Epoch 117/1000
305/305 [==============================] - 0s 80us/step - loss: 0.3267 - accuracy: 0.8426 - val_loss: 0.4201 - val_accuracy: 0.8092
Epoch 118/1000
305/305 [==============================] - 0s 84us/step - loss: 0.3298 - accuracy: 0.8492 - val_loss: 0.3390 - val_acc

Epoch 168/1000
305/305 [==============================] - 0s 89us/step - loss: 0.2996 - accuracy: 0.8492 - val_loss: 0.3070 - val_accuracy: 0.8397
Epoch 169/1000
305/305 [==============================] - 0s 100us/step - loss: 0.3051 - accuracy: 0.8492 - val_loss: 0.3106 - val_accuracy: 0.8397
Epoch 170/1000
305/305 [==============================] - 0s 219us/step - loss: 0.2991 - accuracy: 0.8590 - val_loss: 0.3330 - val_accuracy: 0.8397
Epoch 171/1000
305/305 [==============================] - 0s 89us/step - loss: 0.2986 - accuracy: 0.8525 - val_loss: 0.3189 - val_accuracy: 0.8473
Epoch 172/1000
305/305 [==============================] - 0s 87us/step - loss: 0.2923 - accuracy: 0.8525 - val_loss: 0.3073 - val_accuracy: 0.8321
Epoch 173/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2957 - accuracy: 0.8525 - val_loss: 0.3070 - val_accuracy: 0.8473
Epoch 174/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2919 - accuracy: 0.8590 - val_loss:

Epoch 224/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2799 - accuracy: 0.8525 - val_loss: 0.3260 - val_accuracy: 0.8473
Epoch 225/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2808 - accuracy: 0.8721 - val_loss: 0.3508 - val_accuracy: 0.8321
Epoch 226/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2938 - accuracy: 0.8590 - val_loss: 0.2860 - val_accuracy: 0.8550
Epoch 227/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2747 - accuracy: 0.8590 - val_loss: 0.2883 - val_accuracy: 0.8550
Epoch 228/1000
305/305 [==============================] - 0s 81us/step - loss: 0.2720 - accuracy: 0.8590 - val_loss: 0.3030 - val_accuracy: 0.8473
Epoch 229/1000
305/305 [==============================] - 0s 85us/step - loss: 0.2656 - accuracy: 0.8623 - val_loss: 0.3239 - val_accuracy: 0.8397
Epoch 230/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2842 - accuracy: 0.8590 - val_loss: 0

...

Epoch 727/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2536 - accuracy: 0.8689 - val_loss: 0.2751 - val_accuracy: 0.8473
Epoch 728/1000
305/305 [==============================] - 0s 91us/step - loss: 0.2377 - accuracy: 0.8754 - val_loss: 0.3595 - val_accuracy: 0.8473
Epoch 729/1000
305/305 [==============================] - 0s 85us/step - loss: 0.2246 - accuracy: 0.8787 - val_loss: 0.3103 - val_accuracy: 0.8702
Epoch 730/1000
305/305 [==============================] - 0s 92us/step - loss: 0.2231 - accuracy: 0.8820 - val_loss: 0.3802 - val_accuracy: 0.8550
Epoch 731/1000
305/305 [==============================] - 0s 93us/step - loss: 0.2261 - accuracy: 0.8754 - val_loss: 0.3319 - val_accuracy: 0.8626
Epoch 732/1000
305/305 [==============================] - 0s 91us/step - loss: 0.2246 - accuracy: 0.8885 - val_loss: 0.2737 - val_accuracy: 0.8473
Epoch 733/1000
305/305 [==============================] - 0s 87us/step - loss: 0.2457 - accuracy: 0.8754 - val_loss: 0

Epoch 783/1000
305/305 [==============================] - 0s 83us/step - loss: 0.2442 - accuracy: 0.8721 - val_loss: 0.2888 - val_accuracy: 0.8473
Epoch 784/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2271 - accuracy: 0.8787 - val_loss: 0.3506 - val_accuracy: 0.8626
Epoch 785/1000
305/305 [==============================] - 0s 83us/step - loss: 0.2184 - accuracy: 0.8918 - val_loss: 0.2874 - val_accuracy: 0.8473
Epoch 786/1000
305/305 [==============================] - 0s 89us/step - loss: 0.2371 - accuracy: 0.8754 - val_loss: 0.2866 - val_accuracy: 0.8626
Epoch 787/1000
305/305 [==============================] - 0s 86us/step - loss: 0.2274 - accuracy: 0.8820 - val_loss: 0.3407 - val_accuracy: 0.8626
Epoch 788/1000
305/305 [==============================] - 0s 83us/step - loss: 0.2291 - accuracy: 0.8820 - val_loss: 0.2906 - val_accuracy: 0.8626
Epoch 789/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2187 - accuracy: 0.8787 - val_loss: 0

Epoch 839/1000
305/305 [==============================] - 0s 85us/step - loss: 0.2232 - accuracy: 0.8820 - val_loss: 0.2848 - val_accuracy: 0.8626
Epoch 840/1000
305/305 [==============================] - 0s 85us/step - loss: 0.2269 - accuracy: 0.8787 - val_loss: 0.2748 - val_accuracy: 0.8473
Epoch 841/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2201 - accuracy: 0.8885 - val_loss: 0.3102 - val_accuracy: 0.8550
Epoch 842/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2253 - accuracy: 0.8820 - val_loss: 0.3482 - val_accuracy: 0.8473
Epoch 843/1000
305/305 [==============================] - 0s 81us/step - loss: 0.2214 - accuracy: 0.8885 - val_loss: 0.3618 - val_accuracy: 0.8626
Epoch 844/1000
305/305 [==============================] - 0s 77us/step - loss: 0.2207 - accuracy: 0.8852 - val_loss: 0.3147 - val_accuracy: 0.8702
Epoch 845/1000
305/305 [==============================] - 0s 83us/step - loss: 0.2266 - accuracy: 0.8852 - val_loss: 0

Epoch 895/1000
305/305 [==============================] - 0s 84us/step - loss: 0.2239 - accuracy: 0.8820 - val_loss: 0.2866 - val_accuracy: 0.8626
Epoch 896/1000
305/305 [==============================] - 0s 88us/step - loss: 0.2226 - accuracy: 0.8754 - val_loss: 0.3071 - val_accuracy: 0.8702
Epoch 897/1000
305/305 [==============================] - 0s 80us/step - loss: 0.2255 - accuracy: 0.8787 - val_loss: 0.3024 - val_accuracy: 0.8168
Epoch 898/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2227 - accuracy: 0.8852 - val_loss: 0.3048 - val_accuracy: 0.8702
Epoch 899/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2304 - accuracy: 0.8787 - val_loss: 0.2645 - val_accuracy: 0.8626
Epoch 900/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2301 - accuracy: 0.8787 - val_loss: 0.2742 - val_accuracy: 0.8702
Epoch 901/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2314 - accuracy: 0.8885 - val_loss: 0

Epoch 951/1000
305/305 [==============================] - 0s 82us/step - loss: 0.2187 - accuracy: 0.8885 - val_loss: 0.2848 - val_accuracy: 0.8626
Epoch 952/1000
305/305 [==============================] - 0s 83us/step - loss: 0.2138 - accuracy: 0.8918 - val_loss: 0.3174 - val_accuracy: 0.8168
Epoch 953/1000
305/305 [==============================] - 0s 83us/step - loss: 0.2274 - accuracy: 0.8918 - val_loss: 0.2853 - val_accuracy: 0.8626
Epoch 954/1000
305/305 [==============================] - 0s 86us/step - loss: 0.2175 - accuracy: 0.8885 - val_loss: 0.3358 - val_accuracy: 0.8550
Epoch 955/1000
305/305 [==============================] - 0s 81us/step - loss: 0.2375 - accuracy: 0.8689 - val_loss: 0.2683 - val_accuracy: 0.8855
Epoch 956/1000
305/305 [==============================] - 0s 79us/step - loss: 0.2245 - accuracy: 0.8820 - val_loss: 0.3688 - val_accuracy: 0.8473
Epoch 957/1000
305/305 [==============================] - 0s 83us/step - loss: 0.2297 - accuracy: 0.8852 - val_loss: 0

In [40]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

131/131 [==============================] - 0s 76us/step
over-sampling test accuracy: 83.21%


In [41]:
############ Feature selection using lasso ##########

In [42]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [43]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [44]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [45]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [46]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 974
selected features: 36


In [47]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 26,  32,  42,  76,  83,  84,  85,  86, 135, 137, 162, 191, 224,
        273, 305, 393, 461, 475, 480, 494, 543, 553, 620, 621, 634, 650,
        672, 756, 847, 851, 852, 853, 854, 857, 972, 973]])

In [48]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTAATAGCTAGCACTTAATTGTGTTGGCTATTTTTTATGTCCAAAACGTGCTGATGACATAAAAAGCACGCATGGAAAAACAGTCGACAGACTATAAA',
       'TTTGCCAGTATC', 'TTTCAGCGACT', 'TTGTAAATAAGC', 'TTGCAAATCCAC',
       'TTGATGAAGCAGTAGGG', 'TTGATGAAGCAGTAGGGA',
       'TTGATGAAGCAGTAGGGATTATG', 'TTATCATCAAATG',
       'TTATAGTCTGTCGACTGTTTTTCCATGCGTGCTTTTTATGTCATCAGCACGTTTTGGACATAAAAAATAGCCAACACAATTAAGTGCTAGCTATTAAAAG',
       'TTAATTGAATAACGGGAAGTAGCTCAGCTTGGTAGAGCACTTGGTTTGGGACCAAGGGGTCGCAGGTTCGAATCCTGTCTTCCCGATTACTTCTTAAATT',
       'TGTCTGATTTTT', 'TGCTACCGCA',
       'TCTTTTAATAGCTAGCACTTAATTGTGTTGGCTATTTTTTATGTCCAAAACGTGCTGATGACATAAAAAGCACGCATGGAAAAACAGTCGACAGACTATA',
       'TCGCTGAAATAT', 'TAGATTCAAATAT', 'GTTGCAAATCCAC', 'GTGATGTTGTTTGT',
       'GTGAATTCATG', 'GTAGCTGGATT', 'GCTTGACTGT', 'GCTAATTATGTTTCTGGATT',
       'CTTGATGAAGCAGTAGGG', 'CTTGATGAAGCAGTAGGGATTATG',
       'CTCTTTTAATAGCTAGCACTTAATTGTGTTGGCTATTTTTTATGTCCAAAACGTGCTGATGACATAAAAAGCACGCATGGAAAAACAGTCGACAGACTAT',
       'CTAATGTGGT', 'CGACTAAT

In [49]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            962, 963, 964, 965, 966, 967, 968, 969, 970, 971],
           dtype='int64', length=938)

In [50]:
len(removed_features)

938

In [51]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 36) (255,) (255, 37)


In [52]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 154), (1, 166)]


In [53]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 218), (1, 218)]


In [54]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [55]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [56]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [57]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [58]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 224 samples, validate on 96 samples
Epoch 1/1000
224/224 [==============================] - 0s 464us/step - loss: 11.0661 - accuracy: 0.6071 - val_loss: 5.0449 - val_accuracy: 0.6354
Epoch 2/1000
224/224 [==============================] - 0s 60us/step - loss: 1.0325 - accuracy: 0.8259 - val_loss: 1.3881 - val_accuracy: 0.8229
Epoch 3/1000
224/224 [==============================] - 0s 71us/step - loss: 0.5672 - accuracy: 0.8839 - val_loss: 0.5487 - val_accuracy: 0.7188
Epoch 4/1000
224/224 [==============================] - 0s 80us/step - loss: 0.5038 - accuracy: 0.7991 - val_loss: 0.5367 - val_accuracy: 0.7188
Epoch 5/1000
224/224 [==============================] - 0s 74us/step - loss: 0.4892 - accuracy: 0.8036 - val_loss: 0.5260 - val_accuracy: 0.7396
Epoch 6/1000
224/224 [==============================] - 0s 81us/step - loss: 0.4771 - accuracy: 0.7991 - val_loss: 0.5156 - val_accuracy: 0.7396
Epoch 7/1000
224/224 [==============================] - 0s 84us/step - loss: 0.4639

Epoch 57/1000
224/224 [==============================] - 0s 75us/step - loss: 0.2330 - accuracy: 0.9420 - val_loss: 0.3243 - val_accuracy: 0.8750
Epoch 58/1000
224/224 [==============================] - 0s 70us/step - loss: 0.2321 - accuracy: 0.9420 - val_loss: 0.3236 - val_accuracy: 0.8750
Epoch 59/1000
224/224 [==============================] - 0s 73us/step - loss: 0.2306 - accuracy: 0.9420 - val_loss: 0.3220 - val_accuracy: 0.8750
Epoch 60/1000
224/224 [==============================] - 0s 80us/step - loss: 0.2295 - accuracy: 0.9375 - val_loss: 0.3211 - val_accuracy: 0.8750
Epoch 61/1000
224/224 [==============================] - 0s 72us/step - loss: 0.2279 - accuracy: 0.9420 - val_loss: 0.3201 - val_accuracy: 0.8854
Epoch 62/1000
224/224 [==============================] - 0s 70us/step - loss: 0.2269 - accuracy: 0.9420 - val_loss: 0.3191 - val_accuracy: 0.8854
Epoch 63/1000
224/224 [==============================] - 0s 70us/step - loss: 0.2259 - accuracy: 0.9420 - val_loss: 0.3181 -

Epoch 113/1000
224/224 [==============================] - 0s 93us/step - loss: 0.1904 - accuracy: 0.9420 - val_loss: 0.2898 - val_accuracy: 0.9062
Epoch 114/1000
224/224 [==============================] - 0s 92us/step - loss: 0.1896 - accuracy: 0.9420 - val_loss: 0.2894 - val_accuracy: 0.9062
Epoch 115/1000
224/224 [==============================] - 0s 98us/step - loss: 0.1898 - accuracy: 0.9420 - val_loss: 0.2892 - val_accuracy: 0.9062
Epoch 116/1000
224/224 [==============================] - 0s 125us/step - loss: 0.1888 - accuracy: 0.9420 - val_loss: 0.2887 - val_accuracy: 0.9062
Epoch 117/1000
224/224 [==============================] - 0s 94us/step - loss: 0.1885 - accuracy: 0.9420 - val_loss: 0.2883 - val_accuracy: 0.9062
Epoch 118/1000
224/224 [==============================] - 0s 93us/step - loss: 0.1880 - accuracy: 0.9420 - val_loss: 0.2881 - val_accuracy: 0.9062
Epoch 119/1000
224/224 [==============================] - 0s 99us/step - loss: 0.1875 - accuracy: 0.9420 - val_loss: 

Epoch 169/1000
224/224 [==============================] - 0s 88us/step - loss: 0.1682 - accuracy: 0.9464 - val_loss: 0.2739 - val_accuracy: 0.9062
Epoch 170/1000
224/224 [==============================] - 0s 76us/step - loss: 0.1695 - accuracy: 0.9464 - val_loss: 0.2737 - val_accuracy: 0.9062
Epoch 171/1000
224/224 [==============================] - 0s 66us/step - loss: 0.1678 - accuracy: 0.9464 - val_loss: 0.2735 - val_accuracy: 0.9062
Epoch 172/1000
224/224 [==============================] - 0s 74us/step - loss: 0.1674 - accuracy: 0.9464 - val_loss: 0.2733 - val_accuracy: 0.9062
Epoch 173/1000
224/224 [==============================] - 0s 77us/step - loss: 0.1678 - accuracy: 0.9464 - val_loss: 0.2729 - val_accuracy: 0.9062
Epoch 174/1000
224/224 [==============================] - 0s 73us/step - loss: 0.1669 - accuracy: 0.9464 - val_loss: 0.2728 - val_accuracy: 0.9062
Epoch 175/1000
224/224 [==============================] - 0s 76us/step - loss: 0.1672 - accuracy: 0.9464 - val_loss: 0

Epoch 225/1000
224/224 [==============================] - 0s 75us/step - loss: 0.1534 - accuracy: 0.9464 - val_loss: 0.2641 - val_accuracy: 0.8958
Epoch 226/1000
224/224 [==============================] - 0s 79us/step - loss: 0.1531 - accuracy: 0.9464 - val_loss: 0.2637 - val_accuracy: 0.8958
Epoch 227/1000
224/224 [==============================] - 0s 71us/step - loss: 0.1540 - accuracy: 0.9464 - val_loss: 0.2640 - val_accuracy: 0.8958
Epoch 228/1000
224/224 [==============================] - 0s 73us/step - loss: 0.1530 - accuracy: 0.9464 - val_loss: 0.2638 - val_accuracy: 0.8958
Epoch 229/1000
224/224 [==============================] - 0s 70us/step - loss: 0.1530 - accuracy: 0.9509 - val_loss: 0.2638 - val_accuracy: 0.8958
Epoch 230/1000
224/224 [==============================] - 0s 84us/step - loss: 0.1527 - accuracy: 0.9464 - val_loss: 0.2633 - val_accuracy: 0.8958
Epoch 231/1000
224/224 [==============================] - 0s 75us/step - loss: 0.1518 - accuracy: 0.9509 - val_loss: 0

...

Epoch 727/1000
224/224 [==============================] - 0s 74us/step - loss: 0.0949 - accuracy: 0.9732 - val_loss: 0.2482 - val_accuracy: 0.8958
Epoch 728/1000
224/224 [==============================] - 0s 72us/step - loss: 0.0949 - accuracy: 0.9732 - val_loss: 0.2477 - val_accuracy: 0.8958
Epoch 729/1000
224/224 [==============================] - 0s 78us/step - loss: 0.0949 - accuracy: 0.9732 - val_loss: 0.2476 - val_accuracy: 0.8958
Epoch 730/1000
224/224 [==============================] - 0s 72us/step - loss: 0.0948 - accuracy: 0.9732 - val_loss: 0.2475 - val_accuracy: 0.8958
Epoch 731/1000
224/224 [==============================] - 0s 69us/step - loss: 0.0950 - accuracy: 0.9732 - val_loss: 0.2476 - val_accuracy: 0.8958
Epoch 732/1000
224/224 [==============================] - 0s 70us/step - loss: 0.0950 - accuracy: 0.9732 - val_loss: 0.2481 - val_accuracy: 0.8958
Epoch 733/1000
224/224 [==============================] - 0s 72us/step - loss: 0.0948 - accuracy: 0.9732 - val_loss: 0

Epoch 783/1000
224/224 [==============================] - 0s 70us/step - loss: 0.0928 - accuracy: 0.9732 - val_loss: 0.2490 - val_accuracy: 0.8958
Epoch 784/1000
224/224 [==============================] - 0s 75us/step - loss: 0.0928 - accuracy: 0.9732 - val_loss: 0.2488 - val_accuracy: 0.8958
Epoch 785/1000
224/224 [==============================] - 0s 68us/step - loss: 0.0925 - accuracy: 0.9732 - val_loss: 0.2487 - val_accuracy: 0.8958
Epoch 786/1000
224/224 [==============================] - 0s 74us/step - loss: 0.0924 - accuracy: 0.9732 - val_loss: 0.2490 - val_accuracy: 0.8958
Epoch 787/1000
224/224 [==============================] - 0s 73us/step - loss: 0.0924 - accuracy: 0.9732 - val_loss: 0.2493 - val_accuracy: 0.8958
Epoch 788/1000
224/224 [==============================] - 0s 71us/step - loss: 0.0925 - accuracy: 0.9732 - val_loss: 0.2492 - val_accuracy: 0.8958
Epoch 789/1000
224/224 [==============================] - 0s 69us/step - loss: 0.0924 - accuracy: 0.9732 - val_loss: 0

Epoch 839/1000
224/224 [==============================] - 0s 85us/step - loss: 0.0906 - accuracy: 0.9732 - val_loss: 0.2503 - val_accuracy: 0.8958
Epoch 840/1000
224/224 [==============================] - 0s 71us/step - loss: 0.0906 - accuracy: 0.9732 - val_loss: 0.2506 - val_accuracy: 0.8958
Epoch 841/1000
224/224 [==============================] - 0s 77us/step - loss: 0.0906 - accuracy: 0.9732 - val_loss: 0.2508 - val_accuracy: 0.8958
Epoch 842/1000
224/224 [==============================] - 0s 74us/step - loss: 0.0907 - accuracy: 0.9732 - val_loss: 0.2512 - val_accuracy: 0.8958
Epoch 843/1000
224/224 [==============================] - 0s 75us/step - loss: 0.0904 - accuracy: 0.9732 - val_loss: 0.2508 - val_accuracy: 0.8958
Epoch 844/1000
224/224 [==============================] - 0s 76us/step - loss: 0.0905 - accuracy: 0.9732 - val_loss: 0.2510 - val_accuracy: 0.8958
Epoch 845/1000
224/224 [==============================] - 0s 75us/step - loss: 0.0905 - accuracy: 0.9732 - val_loss: 0

224/224 [==============================] - 0s 63us/step - loss: 0.0887 - accuracy: 0.9732 - val_loss: 0.2499 - val_accuracy: 0.8958
Epoch 895/1000
224/224 [==============================] - 0s 71us/step - loss: 0.0890 - accuracy: 0.9732 - val_loss: 0.2502 - val_accuracy: 0.8958
Epoch 896/1000
224/224 [==============================] - 0s 74us/step - loss: 0.0889 - accuracy: 0.9732 - val_loss: 0.2506 - val_accuracy: 0.8958
Epoch 897/1000
224/224 [==============================] - 0s 77us/step - loss: 0.0889 - accuracy: 0.9732 - val_loss: 0.2513 - val_accuracy: 0.8958
Epoch 898/1000
224/224 [==============================] - 0s 90us/step - loss: 0.0889 - accuracy: 0.9732 - val_loss: 0.2511 - val_accuracy: 0.8958
Epoch 899/1000
224/224 [==============================] - 0s 70us/step - loss: 0.0888 - accuracy: 0.9732 - val_loss: 0.2512 - val_accuracy: 0.8958
Epoch 900/1000
224/224 [==============================] - 0s 74us/step - loss: 0.0887 - accuracy: 0.9732 - val_loss: 0.2517 - val_acc

Epoch 950/1000
224/224 [==============================] - 0s 70us/step - loss: 0.0874 - accuracy: 0.9732 - val_loss: 0.2526 - val_accuracy: 0.9062
Epoch 951/1000
224/224 [==============================] - 0s 74us/step - loss: 0.0873 - accuracy: 0.9732 - val_loss: 0.2527 - val_accuracy: 0.9062
Epoch 952/1000
224/224 [==============================] - 0s 74us/step - loss: 0.0874 - accuracy: 0.9732 - val_loss: 0.2530 - val_accuracy: 0.9062
Epoch 953/1000
224/224 [==============================] - 0s 75us/step - loss: 0.0875 - accuracy: 0.9732 - val_loss: 0.2535 - val_accuracy: 0.9062
Epoch 954/1000
224/224 [==============================] - 0s 74us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.2532 - val_accuracy: 0.9062
Epoch 955/1000
224/224 [==============================] - 0s 71us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.2535 - val_accuracy: 0.9062
Epoch 956/1000
224/224 [==============================] - 0s 68us/step - loss: 0.0873 - accuracy: 0.9732 - val_loss: 0

In [59]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


96/96 [==============================] - 0s 81us/step
combination test accuracy: 90.62%


In [82]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [83]:
model2_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [84]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 224 samples, validate on 96 samples
Epoch 1/1000
224/224 [==============================] - 0s 750us/step - loss: 5.4556 - accuracy: 0.3795 - val_loss: 1.5441 - val_accuracy: 0.3438
Epoch 2/1000
224/224 [==============================] - 0s 68us/step - loss: 2.7351 - accuracy: 0.4911 - val_loss: 0.9282 - val_accuracy: 0.4583
Epoch 3/1000
224/224 [==============================] - 0s 106us/step - loss: 2.0055 - accuracy: 0.5536 - val_loss: 0.8765 - val_accuracy: 0.4688
Epoch 4/1000
224/224 [==============================] - 0s 109us/step - loss: 2.3975 - accuracy: 0.4821 - val_loss: 0.8431 - val_accuracy: 0.4583
Epoch 5/1000
224/224 [==============================] - 0s 101us/step - loss: 2.6248 - accuracy: 0.5223 - val_loss: 0.8102 - val_accuracy: 0.5312
Epoch 6/1000
224/224 [==============================] - 0s 110us/step - loss: 1.6883 - accuracy: 0.5491 - val_loss: 0.7921 - val_accuracy: 0.5729
Epoch 7/1000
224/224 [==============================] - 0s 101us/step - loss: 2.

224/224 [==============================] - 0s 102us/step - loss: 1.4057 - accuracy: 0.8839 - val_loss: 0.5510 - val_accuracy: 0.7708
Epoch 57/1000
224/224 [==============================] - 0s 100us/step - loss: 2.0016 - accuracy: 0.8393 - val_loss: 0.5488 - val_accuracy: 0.7708
Epoch 58/1000
224/224 [==============================] - 0s 102us/step - loss: 1.8633 - accuracy: 0.8616 - val_loss: 0.5416 - val_accuracy: 0.7708
Epoch 59/1000
224/224 [==============================] - 0s 86us/step - loss: 1.7255 - accuracy: 0.8571 - val_loss: 0.5360 - val_accuracy: 0.7708
Epoch 60/1000
224/224 [==============================] - 0s 95us/step - loss: 1.7160 - accuracy: 0.8616 - val_loss: 0.5355 - val_accuracy: 0.7708
Epoch 61/1000
224/224 [==============================] - 0s 78us/step - loss: 2.0404 - accuracy: 0.8438 - val_loss: 0.5308 - val_accuracy: 0.7708
Epoch 62/1000
224/224 [==============================] - 0s 81us/step - loss: 2.0351 - accuracy: 0.8438 - val_loss: 0.5266 - val_accura

Epoch 112/1000
224/224 [==============================] - 0s 93us/step - loss: 1.6680 - accuracy: 0.8616 - val_loss: 0.4251 - val_accuracy: 0.7708
Epoch 113/1000
224/224 [==============================] - 0s 89us/step - loss: 1.5960 - accuracy: 0.8571 - val_loss: 0.4303 - val_accuracy: 0.7708
Epoch 114/1000
224/224 [==============================] - 0s 87us/step - loss: 1.7439 - accuracy: 0.8527 - val_loss: 0.4254 - val_accuracy: 0.7708
Epoch 115/1000
224/224 [==============================] - 0s 79us/step - loss: 1.4859 - accuracy: 0.8571 - val_loss: 0.4297 - val_accuracy: 0.7708
Epoch 116/1000
224/224 [==============================] - 0s 89us/step - loss: 1.8836 - accuracy: 0.8393 - val_loss: 0.4236 - val_accuracy: 0.7708
Epoch 117/1000
224/224 [==============================] - 0s 92us/step - loss: 1.7412 - accuracy: 0.8438 - val_loss: 0.4239 - val_accuracy: 0.7708
Epoch 118/1000
224/224 [==============================] - 0s 84us/step - loss: 1.7383 - accuracy: 0.8527 - val_loss: 0

Epoch 168/1000
224/224 [==============================] - 0s 85us/step - loss: 1.1524 - accuracy: 0.8884 - val_loss: 0.3756 - val_accuracy: 0.9062
Epoch 169/1000
224/224 [==============================] - 0s 95us/step - loss: 2.1104 - accuracy: 0.8304 - val_loss: 0.3710 - val_accuracy: 0.8854
Epoch 170/1000
224/224 [==============================] - 0s 95us/step - loss: 1.3592 - accuracy: 0.8795 - val_loss: 0.3750 - val_accuracy: 0.9062
Epoch 171/1000
224/224 [==============================] - 0s 98us/step - loss: 2.1128 - accuracy: 0.8214 - val_loss: 0.3792 - val_accuracy: 0.9062
Epoch 172/1000
224/224 [==============================] - 0s 83us/step - loss: 2.2379 - accuracy: 0.8259 - val_loss: 0.3769 - val_accuracy: 0.9062
Epoch 173/1000
224/224 [==============================] - 0s 87us/step - loss: 1.3538 - accuracy: 0.8839 - val_loss: 0.3749 - val_accuracy: 0.9062
Epoch 174/1000
224/224 [==============================] - 0s 89us/step - loss: 1.9626 - accuracy: 0.8438 - val_loss: 0

Epoch 224/1000
224/224 [==============================] - 0s 93us/step - loss: 2.0155 - accuracy: 0.8527 - val_loss: 0.3514 - val_accuracy: 0.9062
Epoch 225/1000
224/224 [==============================] - 0s 87us/step - loss: 1.5447 - accuracy: 0.8705 - val_loss: 0.3440 - val_accuracy: 0.9062
Epoch 226/1000
224/224 [==============================] - 0s 87us/step - loss: 1.4616 - accuracy: 0.8884 - val_loss: 0.3540 - val_accuracy: 0.9062
Epoch 227/1000
224/224 [==============================] - 0s 87us/step - loss: 1.8010 - accuracy: 0.8705 - val_loss: 0.3497 - val_accuracy: 0.9062
Epoch 228/1000
224/224 [==============================] - 0s 86us/step - loss: 2.2146 - accuracy: 0.8393 - val_loss: 0.3498 - val_accuracy: 0.9062
Epoch 229/1000
224/224 [==============================] - 0s 88us/step - loss: 1.5316 - accuracy: 0.8795 - val_loss: 0.3458 - val_accuracy: 0.9062
Epoch 230/1000
224/224 [==============================] - 0s 86us/step - loss: 2.0781 - accuracy: 0.8438 - val_loss: 0

...

Epoch 727/1000
224/224 [==============================] - 0s 81us/step - loss: 1.6216 - accuracy: 0.8795 - val_loss: 0.3393 - val_accuracy: 0.8958
Epoch 728/1000
224/224 [==============================] - 0s 97us/step - loss: 1.8335 - accuracy: 0.8661 - val_loss: 0.3397 - val_accuracy: 0.8958
Epoch 729/1000
224/224 [==============================] - 0s 91us/step - loss: 1.7777 - accuracy: 0.8616 - val_loss: 0.3451 - val_accuracy: 0.8958
Epoch 730/1000
224/224 [==============================] - 0s 86us/step - loss: 1.5646 - accuracy: 0.8795 - val_loss: 0.3432 - val_accuracy: 0.8958
Epoch 731/1000
224/224 [==============================] - 0s 95us/step - loss: 1.8380 - accuracy: 0.8616 - val_loss: 0.3376 - val_accuracy: 0.8958
Epoch 732/1000
224/224 [==============================] - 0s 92us/step - loss: 1.5056 - accuracy: 0.8795 - val_loss: 0.3403 - val_accuracy: 0.8958
Epoch 733/1000
224/224 [==============================] - 0s 93us/step - loss: 1.4192 - accuracy: 0.8929 - val_loss: 0

Epoch 783/1000
224/224 [==============================] - 0s 92us/step - loss: 1.6973 - accuracy: 0.8705 - val_loss: 0.3368 - val_accuracy: 0.8958
Epoch 784/1000
224/224 [==============================] - 0s 81us/step - loss: 1.5620 - accuracy: 0.8795 - val_loss: 0.3386 - val_accuracy: 0.8958
Epoch 785/1000
224/224 [==============================] - 0s 87us/step - loss: 1.9030 - accuracy: 0.8571 - val_loss: 0.3386 - val_accuracy: 0.8958
Epoch 786/1000
224/224 [==============================] - 0s 82us/step - loss: 1.4655 - accuracy: 0.8973 - val_loss: 0.3363 - val_accuracy: 0.8958
Epoch 787/1000
224/224 [==============================] - 0s 85us/step - loss: 1.6315 - accuracy: 0.8750 - val_loss: 0.3344 - val_accuracy: 0.8958
Epoch 788/1000
224/224 [==============================] - 0s 82us/step - loss: 1.7483 - accuracy: 0.8750 - val_loss: 0.3367 - val_accuracy: 0.8958
Epoch 789/1000
224/224 [==============================] - 0s 88us/step - loss: 1.4767 - accuracy: 0.8929 - val_loss: 0

Epoch 839/1000
224/224 [==============================] - 0s 83us/step - loss: 1.6959 - accuracy: 0.8705 - val_loss: 0.3388 - val_accuracy: 0.8958
Epoch 840/1000
224/224 [==============================] - 0s 78us/step - loss: 1.6965 - accuracy: 0.8705 - val_loss: 0.3352 - val_accuracy: 0.8958
Epoch 841/1000
224/224 [==============================] - 0s 78us/step - loss: 1.2821 - accuracy: 0.8973 - val_loss: 0.3317 - val_accuracy: 0.8958
Epoch 842/1000
224/224 [==============================] - 0s 78us/step - loss: 1.8267 - accuracy: 0.8661 - val_loss: 0.3321 - val_accuracy: 0.8958
Epoch 843/1000
224/224 [==============================] - 0s 106us/step - loss: 1.5680 - accuracy: 0.8750 - val_loss: 0.3368 - val_accuracy: 0.8958
Epoch 844/1000
224/224 [==============================] - 0s 123us/step - loss: 1.8953 - accuracy: 0.8616 - val_loss: 0.3270 - val_accuracy: 0.9062
Epoch 845/1000
224/224 [==============================] - 0s 112us/step - loss: 1.3963 - accuracy: 0.9018 - val_loss

224/224 [==============================] - 0s 87us/step - loss: 1.6967 - accuracy: 0.8705 - val_loss: 0.3342 - val_accuracy: 0.8958
Epoch 895/1000
224/224 [==============================] - 0s 82us/step - loss: 1.4132 - accuracy: 0.8929 - val_loss: 0.3351 - val_accuracy: 0.8958
Epoch 896/1000
224/224 [==============================] - 0s 101us/step - loss: 1.2063 - accuracy: 0.9062 - val_loss: 0.3361 - val_accuracy: 0.8958
Epoch 897/1000
224/224 [==============================] - 0s 89us/step - loss: 1.8695 - accuracy: 0.8705 - val_loss: 0.3316 - val_accuracy: 0.8958
Epoch 898/1000
224/224 [==============================] - 0s 94us/step - loss: 1.7504 - accuracy: 0.8705 - val_loss: 0.3355 - val_accuracy: 0.8958
Epoch 899/1000
224/224 [==============================] - 0s 83us/step - loss: 1.9056 - accuracy: 0.8527 - val_loss: 0.3365 - val_accuracy: 0.8958
Epoch 900/1000
224/224 [==============================] - 0s 86us/step - loss: 1.5579 - accuracy: 0.8795 - val_loss: 0.3401 - val_ac

Epoch 950/1000
224/224 [==============================] - 0s 75us/step - loss: 2.0167 - accuracy: 0.8571 - val_loss: 0.3377 - val_accuracy: 0.8958
Epoch 951/1000
224/224 [==============================] - 0s 90us/step - loss: 1.1352 - accuracy: 0.9107 - val_loss: 0.3296 - val_accuracy: 0.8958
Epoch 952/1000
224/224 [==============================] - 0s 87us/step - loss: 1.7505 - accuracy: 0.8705 - val_loss: 0.3265 - val_accuracy: 0.8958
Epoch 953/1000
224/224 [==============================] - 0s 80us/step - loss: 1.3421 - accuracy: 0.8973 - val_loss: 0.3327 - val_accuracy: 0.8958
Epoch 954/1000
224/224 [==============================] - 0s 80us/step - loss: 1.8384 - accuracy: 0.8571 - val_loss: 0.3387 - val_accuracy: 0.8958
Epoch 955/1000
224/224 [==============================] - 0s 80us/step - loss: 1.5471 - accuracy: 0.8839 - val_loss: 0.3358 - val_accuracy: 0.8958
Epoch 956/1000
224/224 [==============================] - 0s 87us/step - loss: 2.2911 - accuracy: 0.8393 - val_loss: 0

In [85]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

96/96 [==============================] - 0s 82us/step
combination test accuracy: 89.58%


In [64]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [65]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [66]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/1000
305/305 [==============================] - 0s 349us/step - loss: 12.6891 - accuracy: 0.4984 - val_loss: 0.7311 - val_accuracy: 0.4656
Epoch 2/1000
305/305 [==============================] - 0s 60us/step - loss: 0.7180 - accuracy: 0.4951 - val_loss: 0.8673 - val_accuracy: 0.4733
Epoch 3/1000
305/305 [==============================] - 0s 69us/step - loss: 0.7312 - accuracy: 0.6066 - val_loss: 0.7153 - val_accuracy: 0.5344
Epoch 4/1000
305/305 [==============================] - 0s 69us/step - loss: 0.7028 - accuracy: 0.5934 - val_loss: 0.7061 - val_accuracy: 0.5954
Epoch 5/1000
305/305 [==============================] - 0s 70us/step - loss: 0.6918 - accuracy: 0.6426 - val_loss: 0.6946 - val_accuracy: 0.6183
Epoch 6/1000
305/305 [==============================] - 0s 70us/step - loss: 0.7317 - accuracy: 0.6098 - val_loss: 0.6989 - val_accuracy: 0.5954
Epoch 7/1000
305/305 [==============================] - 0s 73us/step - loss: 0.686

Epoch 57/1000
305/305 [==============================] - 0s 69us/step - loss: 0.5210 - accuracy: 0.7770 - val_loss: 0.5136 - val_accuracy: 0.8015
Epoch 58/1000
305/305 [==============================] - 0s 67us/step - loss: 0.5102 - accuracy: 0.7836 - val_loss: 0.5024 - val_accuracy: 0.8015
Epoch 59/1000
305/305 [==============================] - 0s 69us/step - loss: 0.5023 - accuracy: 0.7869 - val_loss: 0.5088 - val_accuracy: 0.7786
Epoch 60/1000
305/305 [==============================] - 0s 64us/step - loss: 0.6074 - accuracy: 0.7639 - val_loss: 0.5597 - val_accuracy: 0.7710
Epoch 61/1000
305/305 [==============================] - 0s 70us/step - loss: 0.5755 - accuracy: 0.7803 - val_loss: 0.5860 - val_accuracy: 0.7099
Epoch 62/1000
305/305 [==============================] - 0s 67us/step - loss: 0.5855 - accuracy: 0.7475 - val_loss: 0.5408 - val_accuracy: 0.7939
Epoch 63/1000
305/305 [==============================] - 0s 65us/step - loss: 0.5540 - accuracy: 0.7770 - val_loss: 0.5290 -

Epoch 113/1000
305/305 [==============================] - 0s 76us/step - loss: 0.5029 - accuracy: 0.7836 - val_loss: 0.4743 - val_accuracy: 0.8015
Epoch 114/1000
305/305 [==============================] - 0s 65us/step - loss: 0.4859 - accuracy: 0.8000 - val_loss: 0.4728 - val_accuracy: 0.8168
Epoch 115/1000
305/305 [==============================] - 0s 69us/step - loss: 0.4878 - accuracy: 0.8000 - val_loss: 0.4718 - val_accuracy: 0.7939
Epoch 116/1000
305/305 [==============================] - 0s 71us/step - loss: 0.4851 - accuracy: 0.7967 - val_loss: 0.4694 - val_accuracy: 0.8015
Epoch 117/1000
305/305 [==============================] - 0s 67us/step - loss: 0.4712 - accuracy: 0.8000 - val_loss: 0.4877 - val_accuracy: 0.8168
Epoch 118/1000
305/305 [==============================] - 0s 61us/step - loss: 0.4772 - accuracy: 0.8033 - val_loss: 0.4739 - val_accuracy: 0.7939
Epoch 119/1000
305/305 [==============================] - 0s 70us/step - loss: 0.4813 - accuracy: 0.8066 - val_loss: 0

Epoch 169/1000
305/305 [==============================] - 0s 65us/step - loss: 0.4730 - accuracy: 0.8230 - val_loss: 0.4221 - val_accuracy: 0.8168
Epoch 170/1000
305/305 [==============================] - 0s 61us/step - loss: 0.4577 - accuracy: 0.8197 - val_loss: 0.4292 - val_accuracy: 0.8168
Epoch 171/1000
305/305 [==============================] - 0s 73us/step - loss: 0.4379 - accuracy: 0.8230 - val_loss: 0.4517 - val_accuracy: 0.8168
Epoch 172/1000
305/305 [==============================] - 0s 64us/step - loss: 0.4376 - accuracy: 0.8131 - val_loss: 0.4931 - val_accuracy: 0.7328
Epoch 173/1000
305/305 [==============================] - 0s 71us/step - loss: 0.4907 - accuracy: 0.8033 - val_loss: 0.4434 - val_accuracy: 0.8168
Epoch 174/1000
305/305 [==============================] - 0s 67us/step - loss: 0.4293 - accuracy: 0.8164 - val_loss: 0.4186 - val_accuracy: 0.8168
Epoch 175/1000
305/305 [==============================] - 0s 115us/step - loss: 0.4286 - accuracy: 0.8131 - val_loss: 

Epoch 225/1000
305/305 [==============================] - 0s 67us/step - loss: 0.4238 - accuracy: 0.8262 - val_loss: 0.4223 - val_accuracy: 0.8244
Epoch 226/1000
305/305 [==============================] - 0s 65us/step - loss: 0.3961 - accuracy: 0.8328 - val_loss: 0.4034 - val_accuracy: 0.8168
Epoch 227/1000
305/305 [==============================] - 0s 71us/step - loss: 0.4053 - accuracy: 0.8197 - val_loss: 0.4127 - val_accuracy: 0.8168
Epoch 228/1000
305/305 [==============================] - 0s 66us/step - loss: 0.3924 - accuracy: 0.8328 - val_loss: 0.4232 - val_accuracy: 0.8244
Epoch 229/1000
305/305 [==============================] - 0s 74us/step - loss: 0.4093 - accuracy: 0.8230 - val_loss: 0.4232 - val_accuracy: 0.8244
Epoch 230/1000
305/305 [==============================] - 0s 72us/step - loss: 0.4138 - accuracy: 0.8262 - val_loss: 0.3948 - val_accuracy: 0.8168
Epoch 231/1000
305/305 [==============================] - 0s 73us/step - loss: 0.3949 - accuracy: 0.8295 - val_loss: 0

...

Epoch 725/1000
305/305 [==============================] - 0s 74us/step - loss: 0.3052 - accuracy: 0.8689 - val_loss: 0.3388 - val_accuracy: 0.8321
Epoch 726/1000
305/305 [==============================] - 0s 67us/step - loss: 0.3231 - accuracy: 0.8492 - val_loss: 0.3457 - val_accuracy: 0.8244
Epoch 727/1000
305/305 [==============================] - 0s 67us/step - loss: 0.3177 - accuracy: 0.8525 - val_loss: 0.3396 - val_accuracy: 0.8397
Epoch 728/1000
305/305 [==============================] - 0s 65us/step - loss: 0.3090 - accuracy: 0.8656 - val_loss: 0.3518 - val_accuracy: 0.8321
Epoch 729/1000
305/305 [==============================] - 0s 68us/step - loss: 0.3140 - accuracy: 0.8656 - val_loss: 0.3447 - val_accuracy: 0.8321
Epoch 730/1000
305/305 [==============================] - 0s 65us/step - loss: 0.3122 - accuracy: 0.8689 - val_loss: 0.3394 - val_accuracy: 0.8321
Epoch 731/1000
305/305 [==============================] - 0s 67us/step - loss: 0.3120 - accuracy: 0.8656 - val_loss: 0

Epoch 781/1000
305/305 [==============================] - 0s 67us/step - loss: 0.3015 - accuracy: 0.8721 - val_loss: 0.3216 - val_accuracy: 0.8473
Epoch 782/1000
305/305 [==============================] - 0s 67us/step - loss: 0.3062 - accuracy: 0.8623 - val_loss: 0.3237 - val_accuracy: 0.8473
Epoch 783/1000
305/305 [==============================] - 0s 64us/step - loss: 0.3091 - accuracy: 0.8590 - val_loss: 0.3276 - val_accuracy: 0.8473
Epoch 784/1000
305/305 [==============================] - 0s 64us/step - loss: 0.3309 - accuracy: 0.8525 - val_loss: 0.3503 - val_accuracy: 0.8168
Epoch 785/1000
305/305 [==============================] - 0s 72us/step - loss: 0.3148 - accuracy: 0.8590 - val_loss: 0.3373 - val_accuracy: 0.8397
Epoch 786/1000
305/305 [==============================] - 0s 71us/step - loss: 0.3114 - accuracy: 0.8656 - val_loss: 0.3373 - val_accuracy: 0.8244
Epoch 787/1000
305/305 [==============================] - 0s 76us/step - loss: 0.3115 - accuracy: 0.8623 - val_loss: 0

305/305 [==============================] - 0s 65us/step - loss: 0.3120 - accuracy: 0.8656 - val_loss: 0.3460 - val_accuracy: 0.8244
Epoch 837/1000
305/305 [==============================] - 0s 67us/step - loss: 0.3018 - accuracy: 0.8656 - val_loss: 0.3293 - val_accuracy: 0.8397
Epoch 838/1000
305/305 [==============================] - 0s 63us/step - loss: 0.2997 - accuracy: 0.8721 - val_loss: 0.3241 - val_accuracy: 0.8321
Epoch 839/1000
305/305 [==============================] - 0s 65us/step - loss: 0.3095 - accuracy: 0.8623 - val_loss: 0.3204 - val_accuracy: 0.8397
Epoch 840/1000
305/305 [==============================] - 0s 60us/step - loss: 0.3044 - accuracy: 0.8623 - val_loss: 0.3367 - val_accuracy: 0.8321
Epoch 841/1000
305/305 [==============================] - 0s 74us/step - loss: 0.3024 - accuracy: 0.8656 - val_loss: 0.3516 - val_accuracy: 0.8244
Epoch 842/1000
305/305 [==============================] - 0s 66us/step - loss: 0.2936 - accuracy: 0.8721 - val_loss: 0.3230 - val_acc

Epoch 892/1000
305/305 [==============================] - 0s 69us/step - loss: 0.2946 - accuracy: 0.8656 - val_loss: 0.3114 - val_accuracy: 0.8473
Epoch 893/1000
305/305 [==============================] - 0s 65us/step - loss: 0.2935 - accuracy: 0.8623 - val_loss: 0.3113 - val_accuracy: 0.8473
Epoch 894/1000
305/305 [==============================] - 0s 78us/step - loss: 0.2961 - accuracy: 0.8656 - val_loss: 0.3017 - val_accuracy: 0.8473
Epoch 895/1000
305/305 [==============================] - 0s 68us/step - loss: 1.4857 - accuracy: 0.8164 - val_loss: 0.3987 - val_accuracy: 0.8015
Epoch 896/1000
305/305 [==============================] - 0s 71us/step - loss: 0.3538 - accuracy: 0.8328 - val_loss: 0.3813 - val_accuracy: 0.8092
Epoch 897/1000
305/305 [==============================] - 0s 69us/step - loss: 0.3401 - accuracy: 0.8393 - val_loss: 0.3546 - val_accuracy: 0.8397
Epoch 898/1000
305/305 [==============================] - 0s 67us/step - loss: 0.3527 - accuracy: 0.8459 - val_loss: 0

Epoch 948/1000
305/305 [==============================] - 0s 67us/step - loss: 0.3380 - accuracy: 0.8590 - val_loss: 0.3362 - val_accuracy: 0.8473
Epoch 949/1000
305/305 [==============================] - 0s 66us/step - loss: 0.3174 - accuracy: 0.8689 - val_loss: 0.3320 - val_accuracy: 0.8550
Epoch 950/1000
305/305 [==============================] - 0s 71us/step - loss: 0.3170 - accuracy: 0.8689 - val_loss: 0.3315 - val_accuracy: 0.8550
Epoch 951/1000
305/305 [==============================] - 0s 72us/step - loss: 0.3239 - accuracy: 0.8656 - val_loss: 0.3309 - val_accuracy: 0.8550
Epoch 952/1000
305/305 [==============================] - 0s 63us/step - loss: 0.3076 - accuracy: 0.8689 - val_loss: 0.3336 - val_accuracy: 0.8550
Epoch 953/1000
305/305 [==============================] - 0s 73us/step - loss: 0.3148 - accuracy: 0.8623 - val_loss: 0.3379 - val_accuracy: 0.8397
Epoch 954/1000
305/305 [==============================] - 0s 74us/step - loss: 0.3121 - accuracy: 0.8590 - val_loss: 0

In [67]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

131/131 [==============================] - 0s 88us/step
combination test accuracy: 83.97%


In [68]:
############ Logistic Regression ############

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [70]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 97.03%


In [72]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 90.84%


In [74]:
############## Random Forest ##############

In [75]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [76]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 97.03%


In [77]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [78]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 97.71%


In [79]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [80]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.9787234  0.95652174 0.97826087]
0.9827012025901943


In [81]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.93548387 0.98387097 0.8852459  0.96666667 0.96666667]
0.9475868147364711
